In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
import json
from datetime import datetime

class AlmacenGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Sistema de Gestión de Almacenes")
        self.root.geometry("800x600")
        
        # Cargar datos iniciales
        self.cargar_datos()
        
        # Crear pestañas
        self.notebook = ttk.Notebook(root)
        self.notebook.pack(expand=True, fill='both', padx=10, pady=5)
        
        # Pestaña de Visualización
        self.tab_visualizar = ttk.Frame(self.notebook)
        self.notebook.add(self.tab_visualizar, text='Visualizar Inventario')
        
        # Pestaña de Inserción
        self.tab_insertar = ttk.Frame(self.notebook)
        self.notebook.add(self.tab_insertar, text='Insertar Producto')
        
        # Pestaña de Búsqueda
        self.tab_buscar = ttk.Frame(self.notebook)
        self.notebook.add(self.tab_buscar, text='Buscar Producto')
        
        self.setup_visualizar_tab()
        self.setup_insertar_tab()
        self.setup_buscar_tab()
        
    def cargar_datos(self):
        try:
            with open('inventario.txt', 'r') as file:
                contenido = file.read()
                # Convertir el contenido en una lista de diccionarios
                self.inventario = []
                for item in contenido.strip().split('}'):
                    if item.strip():
                        # Asegurarse de que el item termine con }
                        if not item.endswith('}'):
                            item += '}'
                        # Limpiar el item de espacios y caracteres no deseados
                        item = item.strip().lstrip('{').strip()
                        if item:
                            try:
                                self.inventario.append(json.loads('{' + item))
                            except json.JSONDecodeError:
                                continue
        except FileNotFoundError:
            self.inventario = []
            messagebox.showwarning("Advertencia", "No se encontró el archivo inventarios1.txt")
    
    def guardar_datos(self):
        with open('inventario.txt', 'w') as file:
            for item in self.inventario:
                file.write(str(item))
    
    def setup_visualizar_tab(self):
        # Crear Treeview
        columns = ('ID', 'Nombre', 'Tipo', 'Cantidad', 'Fecha Vencimiento')
        self.tree = ttk.Treeview(self.tab_visualizar, columns=columns, show='headings')

        # Configurar columnas
        for col in columns:
            self.tree.heading(col, text=col)
            self.tree.column(col, width=100)

        # Agregar scrollbar
        scrollbar = ttk.Scrollbar(self.tab_visualizar, orient='vertical', command=self.tree.yview)
        self.tree.configure(yscrollcommand=scrollbar.set)

        # Posicionar elementos
        self.tree.pack(side='left', fill='both', expand=True)
        scrollbar.pack(side='right', fill='y')

        # Crear botón para eliminar
        ttk.Button(self.tab_visualizar, text="Eliminar Producto", command=self.eliminar_item).pack(side='bottom', pady=10)

        # Cargar datos en el Treeview
        self.actualizar_treeview()

    def actualizar_treeview(self):
        # Limpiar Treeview
        for item in self.tree.get_children():
            self.tree.delete(item)
        
        # Insertar datos
        for producto in self.inventario:
            self.tree.insert('', 'end', values=(
                producto['id'],
                producto['nombre'],
                producto['tipo'],
                producto['cantidad'],
                producto['fech_venc']
            ))
    
    def setup_insertar_tab(self):
        # Crear formulario
        ttk.Label(self.tab_insertar, text="ID:").grid(row=0, column=0, padx=5, pady=5)
        self.id_entry = ttk.Entry(self.tab_insertar)
        self.id_entry.grid(row=0, column=1, padx=5, pady=5)
        
        ttk.Label(self.tab_insertar, text="Nombre:").grid(row=1, column=0, padx=5, pady=5)
        self.nombre_entry = ttk.Entry(self.tab_insertar)
        self.nombre_entry.grid(row=1, column=1, padx=5, pady=5)
        
        ttk.Label(self.tab_insertar, text="Tipo:").grid(row=2, column=0, padx=5, pady=5)
        self.tipo_combo = ttk.Combobox(self.tab_insertar, values=['perecedero', 'no perecedero'])
        self.tipo_combo.grid(row=2, column=1, padx=5, pady=5)
        
        ttk.Label(self.tab_insertar, text="Cantidad:").grid(row=3, column=0, padx=5, pady=5)
        self.cantidad_entry = ttk.Entry(self.tab_insertar)
        self.cantidad_entry.grid(row=3, column=1, padx=5, pady=5)
        
        ttk.Label(self.tab_insertar, text="Fecha Vencimiento:").grid(row=4, column=0, padx=5, pady=5)
        self.fecha_entry = ttk.Entry(self.tab_insertar)
        self.fecha_entry.grid(row=4, column=1, padx=5, pady=5)
        
        ttk.Button(self.tab_insertar, text="Insertar Producto", command=self.insertar_producto).grid(row=5, column=0, columnspan=2, pady=20)
    
    def insertar_producto(self):
        try:
            nuevo_producto = {
                'id': int(self.id_entry.get()),
                'nombre': self.nombre_entry.get(),
                'tipo': self.tipo_combo.get(),
                'cantidad': int(self.cantidad_entry.get()),
                'fech_venc': self.fecha_entry.get()
            }
            
            # Validaciones básicas
            if not nuevo_producto['nombre'] or not nuevo_producto['tipo']:
                messagebox.showerror("Error", "Todos los campos son obligatorios")
                return
            
            # Verificar si el ID ya existe
            if any(p['id'] == nuevo_producto['id'] for p in self.inventario):
                messagebox.showerror("Error", "El ID ya existe")
                return
            
            self.inventario.append(nuevo_producto)
            self.guardar_datos()
            self.actualizar_treeview()
            
            # Limpiar campos
            self.id_entry.delete(0, 'end')
            self.nombre_entry.delete(0, 'end')
            self.tipo_combo.set('')
            self.cantidad_entry.delete(0, 'end')
            self.fecha_entry.delete(0, 'end')
            
            messagebox.showinfo("Éxito", "Producto insertado correctamente")
            
        except ValueError:
            messagebox.showerror("Error", "Por favor, ingrese valores válidos")
    
    def setup_buscar_tab(self):
        # Crear campos de búsqueda
        ttk.Label(self.tab_buscar, text="Buscar por nombre:").grid(row=0, column=0, padx=5, pady=5)
        self.buscar_nombre_entry = ttk.Entry(self.tab_buscar)
        self.buscar_nombre_entry.grid(row=0, column=1, padx=5, pady=5)
        
        ttk.Label(self.tab_buscar, text="Buscar por tipo:").grid(row=1, column=0, padx=5, pady=5)
        self.buscar_tipo_combo = ttk.Combobox(self.tab_buscar, values=['perecedero', 'no perecedero'])
        self.buscar_tipo_combo.grid(row=1, column=1, padx=5, pady=5)
        ttk.Button(self.tab_buscar, text="Buscar", command=self.buscar_productos).grid(row=2, column=0, columnspan=2, pady=20)

        # Crear Treeview para resultados
        columns = ('ID', 'Nombre', 'Tipo', 'Cantidad', 'Fecha Vencimiento')
        self.tree_busqueda = ttk.Treeview(self.tab_buscar, columns=columns, show='headings')
        
        for col in columns:
            self.tree_busqueda.heading(col, text=col)
            self.tree_busqueda.column(col, width=100)
        
        self.tree_busqueda.grid(row=3, column=0, columnspan=2, padx=5, pady=5)
        
        # Crear botón de eliminación de búsqueda
        ttk.Button(self.tab_buscar, text="Eliminar Producto", command=self.eliminar_item_busqueda).grid(row=4, column=0, columnspan=2, pady=10) 
    
    def eliminar_item(self):
        # Obtener el item seleccionado
        selected_item = self.tree.selection()
        if not selected_item:
            messagebox.showwarning("Advertencia", "Debe seleccionar un producto para eliminar.")
            return
        
        # Eliminar item
        item_values = self.tree.item(selected_item, 'values')
        item_id = int(item_values[0])
        
        self.inventario = [p for p in self.inventario if p['id'] != item_id]
        self.guardar_datos()
        self.actualizar_treeview()
        messagebox.showinfo("Éxito", "Producto eliminado correctamente")
    
    def eliminar_item_busqueda(self):
        # Obtener el item seleccionado
        selected_item = self.tree_busqueda.selection()
        if not selected_item:
            messagebox.showwarning("Advertencia", "Debe seleccionar un producto para eliminar.")
            return
        
        item_values = self.tree_busqueda.item(selected_item, 'values')
        item_id = int(item_values[0])
        
        self.inventario = [p for p in self.inventario if p['id'] != item_id]
        self.guardar_datos()
        self.actualizar_treeview()
        self.buscar_productos()  # Actualizar la búsqueda con los datos restantes
        messagebox.showinfo("Éxito", "Producto eliminado correctamente")
    
    def buscar_productos(self):
    # Limpiar resultados anteriores
        for item in self.tree_busqueda.get_children():
            self.tree_busqueda.delete(item)
    
    # Obtener filtros
        nombre = self.buscar_nombre_entry.get().lower()
        tipo = self.buscar_tipo_combo.get().lower()
    
    # Filtrar productos
        productos_filtrados = [
            p for p in self.inventario
            if (not nombre or nombre in p['nombre'].lower()) and (not tipo or tipo == p['tipo'].lower())
        ]
    
    # Insertar resultados en el Treeview
        for producto in productos_filtrados:
            self.tree_busqueda.insert('', 'end', values=(
                producto['id'],
                producto['nombre'],
                producto['tipo'],
                producto['cantidad'],
                producto['fech_venc']
        ))


if __name__ == "__main__":
    root = tk.Tk()
    app = AlmacenGUI(root)
    root.mainloop()  

In [6]:
import json
from datetime import datetime
from collections import defaultdict

# Clase Producto
class Producto:
    def __init__(self, nombre, fecha_vencimiento=None, perecible=False, cantidad=0, categoria=None):
        self.nombre = nombre
        self.fecha_vencimiento = datetime.strptime(fecha_vencimiento, "%Y-%m-%d") if fecha_vencimiento else None
        self.perecible = perecible
        self.cantidad = int(cantidad)  # Convertir a entero
        self.categoria = categoria
        self.siguiente = None  # Enlace al siguiente producto

# Clase Nodo
class Nodo:
    def __init__(self, tipo):
        self.tipo = tipo
        self.producto = None
        self.hijo = None  # Primer subcategoría o producto
        self.hermano = None  # Siguiente nodo del mismo nivel

    def agregar_producto(self, nuevo_producto):
        if not self.producto:
            self.producto = nuevo_producto
        else:
            actual = self.producto
            anterior = None
            while actual and (not nuevo_producto.perecible or 
                              (nuevo_producto.perecible and nuevo_producto.fecha_vencimiento >= actual.fecha_vencimiento)):
                anterior = actual
                actual = actual.siguiente
            nuevo_producto.siguiente = actual
            if anterior:
                anterior.siguiente = nuevo_producto
            else:
                self.producto = nuevo_producto

    def agregar_hijo(self, nodo_hijo):
        if not self.hijo:
            self.hijo = nodo_hijo
        else:
            actual = self.hijo
            while actual.hermano:
                actual = actual.hermano
            actual.hermano = nodo_hijo

    def buscar_producto_por_tipo(self, tipo):
        if self.tipo == tipo:
            return self
        actual = self.hijo
        while actual:
            resultado = actual.buscar_producto_por_tipo(tipo)
            if resultado:
                return resultado
            actual = actual.hermano
        return None
    
    def buscar_producto_por_vencimiento(self, fecha):
        fecha_busqueda = datetime.strptime(fecha, "%Y-%m-%d")
        producto_actual = self.producto
        while producto_actual:
            if producto_actual.perecible and producto_actual.fecha_vencimiento <= fecha_busqueda:
                print(f"Producto que vence antes del {fecha}: {producto_actual.nombre}")
            producto_actual = producto_actual.siguiente
        hijo = self.hijo
        while hijo:
            hijo.buscar_producto_por_vencimiento(fecha)
            hijo = hijo.hermano

# Clase Arbol
class Arbol:
    def __init__(self, tipo_raiz):
        self.raiz = Nodo(tipo_raiz)

    def agregar_tipo(self, nodo_tipo):
        self.raiz.agregar_hijo(nodo_tipo)

    def agregar_producto_a_tipo(self, tipo, producto):
        nodo_tipo = self.raiz.buscar_producto_por_tipo(tipo)
        if nodo_tipo:
            nodo_tipo.agregar_producto(producto)
        else:
            print(f"Tipo de producto '{tipo}' no encontrado.")
    
    def buscar_por_tipo(self, tipo):
        nodo_tipo = self.raiz.buscar_producto_por_tipo(tipo)
        return nodo_tipo.producto if nodo_tipo else None

    def buscar_por_vencimiento(self, fecha):
        self.raiz.buscar_producto_por_vencimiento(fecha)

    def calcular_suma_cantidad_por_tipo(self):
        cantidades = defaultdict(int)
        self._calcular_suma_cantidad_por_tipo_recursivo(self.raiz, cantidades)
        return dict(cantidades)

    def _calcular_suma_cantidad_por_tipo_recursivo(self, nodo, cantidades):
        if nodo.producto:
            producto_actual = nodo.producto
            while producto_actual:
                cantidades[nodo.tipo] += producto_actual.cantidad
                producto_actual = producto_actual.siguiente
        if nodo.hijo:
            self._calcular_suma_cantidad_por_tipo_recursivo(nodo.hijo, cantidades)
        if nodo.hermano:
            self._calcular_suma_cantidad_por_tipo_recursivo(nodo.hermano, cantidades)

    def calcular_suma_cantidad_por_nombre(self):
        nombres = defaultdict(int)
        self._calcular_suma_cantidad_por_nombre_recursivo(self.raiz, nombres)
        return dict(nombres)

    def _calcular_suma_cantidad_por_nombre_recursivo(self, nodo, nombres):
        if nodo.producto:
            producto_actual = nodo.producto
            while producto_actual:
                nombres[producto_actual.nombre] += producto_actual.cantidad
                producto_actual = producto_actual.siguiente
        if nodo.hijo:
            self._calcular_suma_cantidad_por_nombre_recursivo(nodo.hijo, nombres)
        if nodo.hermano:
            self._calcular_suma_cantidad_por_nombre_recursivo(nodo.hermano, nombres)

    def calcular_suma_cantidad_por_categoria(self):
        categorias = defaultdict(int)
        self._calcular_suma_cantidad_por_categoria_recursivo(self.raiz, categorias)
        return dict(categorias)

    def _calcular_suma_cantidad_por_categoria_recursivo(self, nodo, categorias):
        if nodo.producto:
            producto_actual = nodo.producto
            while producto_actual:
                categorias[producto_actual.categoria] += producto_actual.cantidad
                producto_actual = producto_actual.siguiente
        if nodo.hijo:
            self._calcular_suma_cantidad_por_categoria_recursivo(nodo.hijo, categorias)
        if nodo.hermano:
            self._calcular_suma_cantidad_por_categoria_recursivo(nodo.hermano, categorias)

# Función para cargar datos desde archivo
def cargar_datos_desde_archivo(nombre_archivo, arbol):
    with open(nombre_archivo, 'r') as archivo:
        lineas = archivo.readlines()
        for linea in lineas:
            datos = json.loads(linea.strip())
            tipo = "Perecedero" if datos["tipo"] == "perecedero" else "No perecedero"
            producto = Producto(
                nombre=datos["nombre"],
                fecha_vencimiento=datos.get("fech_venc"),
                perecible=(datos["tipo"] == "perecedero"),
                cantidad=int(datos["cantidad"]),  # Convertir a entero
                categoria=datos["categoria"]
            )
            # Buscar o crear el nodo del tipo correspondiente
            nodo_tipo = arbol.raiz.buscar_producto_por_tipo(tipo)
            if not nodo_tipo:
                nodo_tipo = Nodo(tipo)
                arbol.agregar_tipo(nodo_tipo)
            # Agregar el producto al nodo
            nodo_tipo.agregar_producto(producto)

# Crear el árbol de productos
arbol_productos = Arbol("Almacén")

# Cargar datos desde inventario.txt
cargar_datos_desde_archivo("inventario.txt", arbol_productos)

* Interfaz *

In [8]:
import tkinter as tk
from tkinter import ttk, messagebox
from datetime import datetime

# Asumiendo que las clases Producto, Nodo y Arbol ya están definidas, aquí solo hago los cambios para integrarlos a Tkinter.
class AlmacenGUI:
    def __init__(self, root, arbol):
        self.root = root
        self.root.title("Sistema de Gestión de Almacenes")
        self.root.geometry("800x600")
        
        # El árbol de productos
        self.arbol = arbol

        # Crear pestañas
        self.notebook = ttk.Notebook(root)
        self.notebook.pack(expand=True, fill='both', padx=10, pady=5)
        
        # Pestaña de Visualización
        self.tab_visualizar = ttk.Frame(self.notebook)
        self.notebook.add(self.tab_visualizar, text='Visualizar Inventario')
        
        # Pestaña de Inserción
        self.tab_insertar = ttk.Frame(self.notebook)
        self.notebook.add(self.tab_insertar, text='Insertar Producto')
        
        # Pestaña de Búsqueda
        self.tab_buscar = ttk.Frame(self.notebook)
        self.notebook.add(self.tab_buscar, text='Buscar Producto')
        
        self.setup_visualizar_tab()
        self.setup_insertar_tab()
        self.setup_buscar_tab()
        
        # Cargar los productos en el árbol a partir de los datos de ejemplo
        self.cargar_datos()

    def cargar_datos(self):
        # Datos de ejemplo (normalmente cargados desde inventario.txt)
        productos = [
            {"id": 1, "nombre": "Manzana", "tipo": "perecedero", "categoria": "fruta", "cantidad": 50, "fech_venc": "2024-10-26"},
            {"id": 2, "nombre": "Arroz", "tipo": "no perecedero", "categoria": "grano", "cantidad": 100, "fech_venc": "2030-01-20"},
            {"id": 3, "nombre": "Leche", "tipo": "perecedero", "categoria": "lacteo", "cantidad": 30, "fech_venc": "2025-11-10"},
            {"id": 4, "nombre": "Lentejas", "tipo": "no perecedero", "categoria": "grano", "cantidad": 80, "fech_venc": "2024-10-26"},
            {"id": 5, "nombre": "Pollo", "tipo": "perecedero", "categoria": "carnes", "cantidad": 20, "fech_venc": "2024-10-27"},
            {"id": 6, "nombre": "Pasta", "tipo": "no perecedero", "categoria": "pasta", "cantidad": 60, "fech_venc": "2027-12-12"},
            {"id": 7, "nombre": "Platano", "tipo": "perecedero", "categoria": "fruta", "cantidad": 55, "fech_venc": "2025-01-12"},
            {"id": 8, "nombre": "Naranja", "tipo": "perecedero", "categoria": "fruta", "cantidad": 2, "fech_venc": "2024-11-15"}
        ]

        for producto_data in productos:
            # Crear producto y agregarlo al árbol
            producto = Producto(
                nombre=producto_data["nombre"],
                fecha_vencimiento=producto_data["fech_venc"],
                perecible=(producto_data["tipo"] == "perecedero"),
                cantidad=producto_data["cantidad"],
                categoria=producto_data["categoria"]
            )
            tipo = "Perecedero" if producto.perecible else "No perecedero"
            nodo_tipo = self.arbol.raiz.buscar_producto_por_tipo(tipo)
            if not nodo_tipo:
                nodo_tipo = Nodo(tipo)
                self.arbol.agregar_tipo(nodo_tipo)
            nodo_tipo.agregar_producto(producto)

    def setup_visualizar_tab(self):
        # Crear Treeview
        columns = ('ID', 'Nombre', 'Tipo', 'Cantidad', 'Fecha Vencimiento')
        self.tree = ttk.Treeview(self.tab_visualizar, columns=columns, show='headings')

        # Configurar columnas
        for col in columns:
            self.tree.heading(col, text=col)
            self.tree.column(col, width=100)

        # Agregar scrollbar
        scrollbar = ttk.Scrollbar(self.tab_visualizar, orient='vertical', command=self.tree.yview)
        self.tree.configure(yscrollcommand=scrollbar.set)

        # Posicionar elementos
        self.tree.pack(side='left', fill='both', expand=True)
        scrollbar.pack(side='right', fill='y')

        # Crear botón para eliminar
        ttk.Button(self.tab_visualizar, text="Eliminar Producto", command=self.eliminar_item).pack(side='bottom', pady=10)

        # Cargar datos en el Treeview
        self.actualizar_treeview()

    def actualizar_treeview(self):
        # Limpiar Treeview
        for item in self.tree.get_children():
            self.tree.delete(item)
        
        # Insertar productos del árbol en el Treeview
        for tipo_nodo in self.arbol.raiz.hijo:
            producto_actual = tipo_nodo.producto
            while producto_actual:
                self.tree.insert('', 'end', values=(
                    producto_actual.id,
                    producto_actual.nombre,
                    tipo_nodo.tipo,
                    producto_actual.cantidad,
                    producto_actual.fecha_vencimiento.strftime("%Y-%m-%d")
                ))
                producto_actual = producto_actual.siguiente
    
    def setup_insertar_tab(self):
        # Crear formulario
        ttk.Label(self.tab_insertar, text="ID:").grid(row=0, column=0, padx=5, pady=5)
        self.id_entry = ttk.Entry(self.tab_insertar)
        self.id_entry.grid(row=0, column=1, padx=5, pady=5)
        
        ttk.Label(self.tab_insertar, text="Nombre:").grid(row=1, column=0, padx=5, pady=5)
        self.nombre_entry = ttk.Entry(self.tab_insertar)
        self.nombre_entry.grid(row=1, column=1, padx=5, pady=5)
        
        ttk.Label(self.tab_insertar, text="Tipo:").grid(row=2, column=0, padx=5, pady=5)
        self.tipo_combo = ttk.Combobox(self.tab_insertar, values=['perecedero', 'no perecedero'])
        self.tipo_combo.grid(row=2, column=1, padx=5, pady=5)
        
        ttk.Label(self.tab_insertar, text="Cantidad:").grid(row=3, column=0, padx=5, pady=5)
        self.cantidad_entry = ttk.Entry(self.tab_insertar)
        self.cantidad_entry.grid(row=3, column=1, padx=5, pady=5)
        
        ttk.Label(self.tab_insertar, text="Fecha Vencimiento:").grid(row=4, column=0, padx=5, pady=5)
        self.fecha_entry = ttk.Entry(self.tab_insertar)
        self.fecha_entry.grid(row=4, column=1, padx=5, pady=5)
        
        ttk.Button(self.tab_insertar, text="Insertar Producto", command=self.insertar_producto).grid(row=5, column=0, columnspan=2, pady=20)
    
    def insertar_producto(self):
        try:
            nuevo_producto = Producto(
                nombre=self.nombre_entry.get(),
                fecha_vencimiento=self.fecha_entry.get(),
                perecible=(self.tipo_combo.get() == "perecedero"),
                cantidad=int(self.cantidad_entry.get()),
                categoria=""
            )
            
            # Validaciones básicas
            if not nuevo_producto.nombre or not nuevo_producto.fecha_vencimiento:
                messagebox.showerror("Error", "Todos los campos son obligatorios")
                return
            
            tipo = "Perecedero" if nuevo_producto.perecible else "No perecedero"
            nodo_tipo = self.arbol.raiz.buscar_producto_por_tipo(tipo)
            if not nodo_tipo:
                nodo_tipo = Nodo(tipo)
                self.arbol.agregar_tipo(nodo_tipo)

            nodo_tipo.agregar_producto(nuevo_producto)
            self.actualizar_treeview()
            messagebox.showinfo("Éxito", "Producto insertado correctamente")
        except ValueError:
            messagebox.showerror("Error", "Por favor, ingrese valores válidos")
    
    def eliminar_item(self):
        selected_item = self.tree.selection()
        if not selected_item:
            messagebox.showerror("Error", "Debe seleccionar un producto para eliminar")
            return
        item_values = self.tree.item(selected_item)['values']
        producto_id = item_values[0]

        # Buscar y eliminar el producto
        for tipo_nodo in self.arbol.raiz.hijo:
            tipo_nodo.eliminar_producto_por_id(producto_id)
        
        self.actualizar_treeview()
        messagebox.showinfo("Éxito", f"Producto con ID {producto_id} eliminado")

    def setup_buscar_tab(self):
        # Crear formulario de búsqueda
        ttk.Label(self.tab_buscar, text="Buscar por ID:").grid(row=0, column=0, padx=5, pady=5)
        self.buscar_id_entry = ttk.Entry(self.tab_buscar)
        self.buscar_id_entry.grid(row=0, column=1, padx=5, pady=5)
        ttk.Button(self.tab_buscar, text="Buscar", command=self.buscar_producto).grid(row=1, column=0, columnspan=2, pady=20)

    def buscar_producto(self):
        producto_id = self.buscar_id_entry.get()
        if not producto_id:
            messagebox.showerror("Error", "Por favor ingrese un ID para buscar")
            return
        
        producto_encontrado = None
        for tipo_nodo in self.arbol.raiz.hijo:
            producto_encontrado = tipo_nodo.buscar_producto_por_id(int(producto_id))
            if producto_encontrado:
                break
        
        if producto_encontrado:
            messagebox.showinfo("Producto encontrado", f"Producto: {producto_encontrado.nombre}, ID: {producto_encontrado.id}")
        else:
            messagebox.showinfo("No encontrado", "Producto no encontrado")
        
# Crear ventana principal
root = tk.Tk()
arbol = Arbol('Productos')  # Crear el árbol que manejará los productos
app = AlmacenGUI(root, arbol)
root.mainloop()


TypeError: 'NoneType' object is not iterable

In [4]:
import tkinter as tk
from tkinter import ttk, messagebox
from datetime import datetime
from collections import defaultdict

# Aquí asumo que ya tienes las clases Producto, Nodo y Arbol definidas.

class AlmacenGUI:
    def __init__(self, root, arbol):
        self.root = root
        self.root.title("Sistema de Gestión de Almacenes")
        self.root.geometry("800x600")
        
        # Inicializar el árbol
        self.arbol = arbol
        
        # Crear pestañas
        self.notebook = ttk.Notebook(root)
        self.notebook.pack(expand=True, fill='both', padx=10, pady=5)
        
        # Pestaña de Visualización
        self.tab_visualizar = ttk.Frame(self.notebook)
        self.notebook.add(self.tab_visualizar, text='Visualizar Inventario')
        
        # Pestaña de Inserción
        self.tab_insertar = ttk.Frame(self.notebook)
        self.notebook.add(self.tab_insertar, text='Insertar Producto')
        
        # Pestaña de Búsqueda
        self.tab_buscar = ttk.Frame(self.notebook)
        self.notebook.add(self.tab_buscar, text='Buscar Producto')
        
        self.setup_visualizar_tab()
        self.setup_insertar_tab()
        self.setup_buscar_tab()
        
    def setup_visualizar_tab(self):
        # Crear Treeview
        columns = ('Nombre', 'Tipo', 'Cantidad', 'Fecha Vencimiento', 'Categoría')
        self.tree = ttk.Treeview(self.tab_visualizar, columns=columns, show='headings')

        # Configurar columnas
        for col in columns:
            self.tree.heading(col, text=col)
            self.tree.column(col, width=120)

        # Agregar scrollbar
        scrollbar = ttk.Scrollbar(self.tab_visualizar, orient='vertical', command=self.tree.yview)
        self.tree.configure(yscrollcommand=scrollbar.set)

        # Posicionar elementos
        self.tree.pack(side='left', fill='both', expand=True)
        scrollbar.pack(side='right', fill='y')

        # Crear botón para eliminar
        ttk.Button(self.tab_visualizar, text="Eliminar Producto", command=self.eliminar_item).pack(side='bottom', pady=10)

        # Cargar datos en el Treeview
        self.actualizar_treeview()

    def actualizar_treeview(self):
        # Limpiar Treeview
        for item in self.tree.get_children():
            self.tree.delete(item)
        
        # Insertar datos desde el árbol
        self.mostrar_productos(self.arbol.raiz)

    def mostrar_productos(self, nodo):
        if nodo.producto:
            actual = nodo.producto
            while actual:
                self.tree.insert('', 'end', values=(
                    actual.nombre,
                    "Perecedero" if actual.perecible else "No perecedero",
                    actual.cantidad,
                    actual.fecha_vencimiento.strftime("%Y-%m-%d") if actual.fecha_vencimiento else "N/A",
                    actual.categoria
                ))
                actual = actual.siguiente
        
        if nodo.hijo:
            self.mostrar_productos(nodo.hijo)
        if nodo.hermano:
            self.mostrar_productos(nodo.hermano)

    def setup_insertar_tab(self):
        # Crear formulario
        ttk.Label(self.tab_insertar, text="Nombre:").grid(row=0, column=0, padx=5, pady=5)
        self.nombre_entry = ttk.Entry(self.tab_insertar)
        self.nombre_entry.grid(row=0, column=1, padx=5, pady=5)
        
        ttk.Label(self.tab_insertar, text="Tipo:").grid(row=1, column=0, padx=5, pady=5)
        self.tipo_combo = ttk.Combobox(self.tab_insertar, values=['perecedero', 'no perecedero'])
        self.tipo_combo.grid(row=1, column=1, padx=5, pady=5)
        
        ttk.Label(self.tab_insertar, text="Cantidad:").grid(row=2, column=0, padx=5, pady=5)
        self.cantidad_entry = ttk.Entry(self.tab_insertar)
        self.cantidad_entry.grid(row=2, column=1, padx=5, pady=5)
        
        ttk.Label(self.tab_insertar, text="Fecha Vencimiento (YYYY-MM-DD):").grid(row=3, column=0, padx=5, pady=5)
        self.fecha_entry = ttk.Entry(self.tab_insertar)
        self.fecha_entry.grid(row=3, column=1, padx=5, pady=5)
        
        ttk.Label(self.tab_insertar, text="Categoría:").grid(row=4, column=0, padx=5, pady=5)
        self.categoria_entry = ttk.Entry(self.tab_insertar)
        self.categoria_entry.grid(row=4, column=1, padx=5, pady=5)
        
        ttk.Button(self.tab_insertar, text="Insertar Producto", command=self.insertar_producto).grid(row=5, column=0, columnspan=2, pady=20)
    
    def insertar_producto(self):
        try:
            nombre = self.nombre_entry.get()
            tipo = self.tipo_combo.get()
            cantidad = int(self.cantidad_entry.get())
            fecha_vencimiento = self.fecha_entry.get()
            categoria = self.categoria_entry.get()
            
            # Validaciones básicas
            if not nombre or not tipo:
                messagebox.showerror("Error", "Todos los campos son obligatorios")
                return
            
            # Crear producto
            producto = Producto(
                nombre=nombre,
                fecha_vencimiento=fecha_vencimiento if tipo == "perecedero" else None,
                perecible=(tipo == "perecedero"),
                cantidad=cantidad,
                categoria=categoria
            )
            
            # Agregar al árbol según el tipo
            tipo_producto = "Perecedero" if producto.perecible else "No perecedero"
            self.arbol.agregar_producto_a_tipo(tipo_producto, producto)
            
            # Limpiar campos y actualizar vista
            self.nombre_entry.delete(0, 'end')
            self.tipo_combo.set('')
            self.cantidad_entry.delete(0, 'end')
            self.fecha_entry.delete(0, 'end')
            self.categoria_entry.delete(0, 'end')
            self.actualizar_treeview()
            
            messagebox.showinfo("Éxito", "Producto insertado correctamente")
            
        except ValueError:
            messagebox.showerror("Error", "Por favor, ingrese valores válidos")

    def setup_buscar_tab(self):
        # Crear campos de búsqueda
        ttk.Label(self.tab_buscar, text="Buscar por nombre:").grid(row=0, column=0, padx=5, pady=5)
        self.buscar_nombre_entry = ttk.Entry(self.tab_buscar)
        self.buscar_nombre_entry.grid(row=0, column=1, padx=5, pady=5)
        
        ttk.Label(self.tab_buscar, text="Buscar por tipo:").grid(row=1, column=0, padx=5, pady=5)
        self.buscar_tipo_combo = ttk.Combobox(self.tab_buscar, values=['perecedero', 'no perecedero'])
        self.buscar_tipo_combo.grid(row=1, column=1, padx=5, pady=5)
        ttk.Button(self.tab_buscar, text="Buscar", command=self.buscar_productos).grid(row=2, column=0, columnspan=2, pady=20)

        # Crear Treeview para resultados
        columns = ('Nombre', 'Tipo', 'Cantidad', 'Fecha Vencimiento', 'Categoría')
        self.tree_busqueda = ttk.Treeview(self.tab_buscar, columns=columns, show='headings')
        
        for col in columns:
            self.tree_busqueda.heading(col, text=col)
            self.tree_busqueda.column(col, width=120)
        
        self.tree_busqueda.grid(row=3, column=0, columnspan=2, padx=5, pady=5)
        
        # Crear botón de eliminación de búsqueda
        ttk.Button(self.tab_buscar, text="Eliminar Producto", command=self.eliminar_item_busqueda).grid(row=4, column=0, columnspan=2, pady=10) 

    def buscar_productos(self):
        # Limpiar resultados anteriores
        for item in self.tree_busqueda.get_children():
            self.tree_busqueda.delete(item)
    
        # Obtener filtros
        nombre = self.buscar_nombre_entry.get().lower()
        tipo = self.buscar_tipo_combo.get().lower()
    
        # Filtrar productos en el árbol
        productos_filtrados = self.filtrar_productos_arbol(self.arbol.raiz, nombre, tipo)
    
        # Insertar resultados en el Treeview
        for producto in productos_filtrados:
            self.tree_busqueda.insert('', 'end', values=(
                producto.nombre,
                "Perecedero" if producto.perecible else "No perecedero",
                producto.cantidad,
                producto.fecha_vencimiento.strftime("%Y-%m-%d") if producto.fecha_vencimiento else "N/A",
                producto.categoria
            ))

    def filtrar_productos_arbol(self, nodo, nombre, tipo):
        productos = []
        if nodo.producto:
            actual = nodo.producto
            while actual:
                if (not nombre or nombre in actual.nombre.lower()) and (not tipo or (tipo == "perecedero" and actual.perecible) or (tipo == "no perecedero" and not actual.perecible)):
                    productos.append(actual)
                actual = actual.siguiente
        
        if nodo.hijo:
            productos.extend(self.filtrar_productos_arbol(nodo.hijo, nombre, tipo))
        if nodo.hermano:
            productos.extend(self.filtrar_productos_arbol(nodo.hermano, nombre, tipo))
        
        return productos

    def eliminar_item(self):
        # Obtener el item seleccionado
        selected_item = self.tree.selection()
        if not selected_item:
            messagebox.showwarning("Advertencia", "Debe seleccionar un producto para eliminar.")
            return
        
        # Eliminar item del árbol
        item_values = self.tree.item(selected_item, 'values')
        nombre = item_values[0]
        tipo = item_values[1].lower()
        
        self.eliminar_producto_del_arbol(self.arbol.raiz, nombre, tipo == "perecedero")
        self.actualizar_treeview()
        messagebox.showinfo("Éxito", "Producto eliminado correctamente")
    
    def eliminar_item_busqueda(self):
        # Obtener el item seleccionado
        selected_item = self.tree_busqueda.selection()
        if not selected_item:
            messagebox.showwarning("Advertencia", "Debe seleccionar un producto para eliminar.")
            return
        
        item_values = self.tree_busqueda.item(selected_item, 'values')
        nombre = item_values[0]
        tipo = item_values[1].lower()
        
        self.eliminar_producto_del_arbol(self.arbol.raiz, nombre, tipo == "perecedero")
        self.actualizar_treeview()
        self.buscar_productos()  # Actualizar la búsqueda con los datos restantes
        messagebox.showinfo("Éxito", "Producto eliminado correctamente")
    
    def eliminar_producto_del_arbol(self, nodo, nombre, perecible):
        if nodo.producto:
            actual = nodo.producto
            previo = None
            while actual:
                if actual.nombre == nombre and actual.perecible == perecible:
                    if previo:
                        previo.siguiente = actual.siguiente
                    else:
                        nodo.producto = actual.siguiente
                    return
                previo = actual
                actual = actual.siguiente
        
        if nodo.hijo:
            self.eliminar_producto_del_arbol(nodo.hijo, nombre, perecible)
        if nodo.hermano:
            self.eliminar_producto_del_arbol(nodo.hermano, nombre, perecible)


if __name__ == "__main__":
    root = tk.Tk()
    
    # Crear el árbol para gestionar los productos
    arbol = Arbol()
    
    app = AlmacenGUI(root, arbol)
    root.mainloop()


TypeError: Arbol.__init__() missing 1 required positional argument: 'tipo_raiz'